# Funcion get_darwins_info
No recibe parametros
Es necesario rellenar la variable 'token' con la API token de Darwinex, que se consigue desde la plataforma de inversor con un simple click.
Esta función debe hacer múltiples llamadas porque las respuestas de la API dan por defecto la info de los darwins en paquetes de 50. Asi que de ahi viene el sleep, tanto en esta función como en tantas otras que recopilen información que requiera hacer varias llamadas a la API.
La función devuelve una variable tipo lista


In [ ]:
import requests
import time

token = ''

def get_darwins_info():
    index = 0
    url = 'https://api.darwinex.com/darwininfo/1.5/products?page=' + str(index)
    headers = {'Authorization': 'Bearer ' + str(token)}
    r = requests.get(url, headers=headers)
    is_last = r.json()['last']
    s = r.json()['content']
    index = index + 1
    print('(',index,') Procesando darwins ',r.json()['content'][0]['productName'],' a ', r.json()['content'][49]['productName'])

    while is_last != True:
        url = 'https://api.darwinex.com/darwininfo/1.5/products?page=' + str(index)
        r = requests.get(url, headers=headers)
        is_last = r.json()['last']
        s.extend(r.json()['content'])
        print('(',index,') Procesando darwins ',r.json()['content'][0]['productName'],' a ', r.json()['content'][len(r.json()['content'])-1]['productName'])
        if index % 10 == 0:
            print('Durmiendo 60 segundos...')
            time.sleep(60)
        index = index + 1
    print('Descarga de Darwins finalizada')
    return(s)


In [ ]:
darwins = get_darwins_info()

In [ ]:
def get_badges(darwins):
    headers = {'Authorization': 'Bearer ' + str(token)}

    i = 1
    for darwin in darwins:
        print('(',i,') Darwin:',darwin['shortName'])
        url = 'https://api.darwinex.com/darwininfo/1.5/products/' + darwin['productName'] + '/history/badges'
        r = requests.get(url, headers=headers)
        darwins[i-1].update({'badges' : r.json()})
        if i % 10 == 0:
            print('Durmiendo 60 segundos...')
            time.sleep(60)
        i = i+1
    return(darwins)